In [27]:
import pandas as pd
qb_df = pd.read_csv('/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_Player_games/cleaned_qb_logs.csv')
rb_df = pd.read_csv('/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_Player_games/cleaned_rb_logs.csv')
wr_df = pd.read_csv('/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_Player_games/cleaned_wr_logs.csv')
te_df = pd.read_csv('/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_Player_games/cleaned_te_logs.csv')

/var/folders/5b/zyfxmg1n27sc10rqxm40tr3r0000gn/T/ipykernel_96902/3569209882.py:4: DtypeWarning: Columns (38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  wr_df = pd.read_csv('/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_Player_games/cleaned_wr_logs.csv')


In [28]:
player_cols = [df[['Player']] for df in (qb_df, rb_df, wr_df, te_df)]
player_df = pd.concat(player_cols, ignore_index=True)
player_df = player_df.drop_duplicates()
player_df = player_df.reset_index()
player_df = player_df.drop(columns=['index'])
print(player_df)


               Player
0         A.J. Feeley
1       A.J. McCarron
2        Aaron Brooks
3       Aaron Rodgers
4     Aidan O'Connell
...               ...
2909     Zach Sudfeld
2910   Zachary Hilton
2911       Zack Kuntz
2912    Zack Pianalto
2913  Zeron Flemister

[2914 rows x 1 columns]


In [1]:
import os
import time
import random
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd

def fetch_url(url):
    """
    GET the URL, return text or None on 404; retry on other errors.
    """
    while True:
        try:
            r = requests.get(url)
            if r.status_code == 404:
                return None
            r.raise_for_status()
            return r.text
        except requests.exceptions.HTTPError as e:
            print(f"HTTP {e.response.status_code} error for {url}, retrying in 4s…")
            time.sleep(4)
        except Exception as e:
            print(f"Network error {e} for {url}, retrying in 4s…")
            time.sleep(4)

def parse_html_table(table_tag):
    """
    Parse a <table> with multi-row headers into a DataFrame:
    - Flatten two header rows (“Receiving” + “Yds” → “Receiving Yds”)
    - Skip any <tr class="thead"> in the <tbody>.
    """
    hdr = table_tag.find("thead")
    rows = hdr.find_all("tr") if hdr else []
    if len(rows) >= 2:
        grp = [th.get_text(strip=True) for th in rows[0].find_all("th")]
        sub = [th.get_text(strip=True) for th in rows[1].find_all("th")]
        headers, idx = [], 0
        for text, th in zip(grp, rows[0].find_all("th")):
            span = int(th.get("colspan", 1))
            for _ in range(span):
                part = sub[idx]
                headers.append(f"{text} {part}" if text else part)
                idx += 1
    elif len(rows) == 1:
        headers = [th.get_text(strip=True) for th in rows[0].find_all("th")]
    else:
        headers = None

    data, body = [], table_tag.find("tbody") or table_tag
    for tr in body.find_all("tr"):
        if tr.get("class") and "thead" in tr.get("class"):
            continue
        cells = [td.get_text(strip=True) for td in tr.find_all(["th","td"])]
        if headers and len(cells) != len(headers):
            continue
        data.append(cells)

    return pd.DataFrame(data, columns=headers)

def flush_pos_groups_to_csv(pos_groups, output_dir):
    """
    Write pos_groups (pos -> list of DataFrames) out to {output_dir}/{pos}.csv.
    """
    os.makedirs(output_dir, exist_ok=True)
    for pos, dfs in pos_groups.items():
        if not dfs:
            print(f"No data for {pos}, skipping.")
            continue
        # collect superset of columns
        all_cols = []
        for df in dfs:
            for c in df.columns:
                if c not in all_cols:
                    all_cols.append(c)
        # reindex, concat
        normalized = [df.reindex(columns=all_cols, fill_value="") for df in dfs]
        combined = pd.concat(normalized, ignore_index=True)
        out_path = os.path.join(output_dir, f"{pos}.csv")
        combined.to_csv(out_path, index=False)
        print(f"Wrote {out_path}: {combined.shape[0]} rows × {combined.shape[1]} cols")

def main():
    base_dir = "/Users/sethfried/Fantasy Football/Cleaned Data/Rookie_college_stats"
    for year in range(2013, 2021):  # 2013–2020 inclusive
        print(f"\n=== Processing rookies for {year} ===")
        output_dir = os.path.join(base_dir, f"{year}_rookie_college_stats")
        pos_groups = {}

        # 1) Fetch and unwrap the draft table
        draft_url = f"https://www.pro-football-reference.com/years/{year}/draft.htm"
        print(f"Fetching draft page for {year}…")
        txt = fetch_url(draft_url)
        if not txt:
            print(f"  ⚠ Could not load draft page for {year}, skipping")
            continue

        soup = BeautifulSoup(txt, "html.parser")
        table = soup.find("table", id="drafts")
        if table is None:
            for comment in soup.find_all(string=lambda t: isinstance(t, Comment)):
                if 'id="drafts"' in comment:
                    table = BeautifulSoup(comment, "html.parser")\
                                .find("table", id="drafts")
                    break
        if table is None:
            print(f"  ⚠ Draft table not found for {year}, skipping")
            continue

        # 2) Identify last column's data-stat
        hdr_rows = table.find("thead").find_all("tr")
        last_stat = hdr_rows[1].find_all("th")[-1].get("data-stat")
        print(f"  Using column data-stat='{last_stat}' for links")

        # 3) Build list of (player_name, link_url)
        links = []
        for tr in table.find("tbody").find_all("tr"):
            if tr.get("class") and "thead" in tr.get("class"):
                continue
            td = tr.find("td", {"data-stat": last_stat})
            if not td:
                continue
            a = td.find("a")
            if not a or not a.get("href"):
                continue
            href  = a["href"]
            url   = href if href.startswith("http") else "https://www.pro-football-reference.com" + href
            player = tr.find("td", {"data-stat":"player"}).get_text(strip=True)
            links.append((player, url))

        # 4) Scrape each player's table and group by Pos
        for player, url in links:
            wait = random.uniform(5, 6)
            print(f"  Waiting {wait:.1f}s before fetching {player!r}")
            time.sleep(wait)

            txt_p = fetch_url(url)
            if not txt_p:
                print(f"    ⚠ skipped {player!r} (page not found)")
                continue

            page_soup = BeautifulSoup(txt_p, "html.parser")
            tbl = page_soup.find("table")
            if not tbl:
                print(f"    ✗ no table on page for {player!r}")
                continue

            df = parse_html_table(tbl)
            df["Player"] = player

            # determine Pos, override Travis Hunter
            pos = df["Pos"].iloc[0] if "Pos" in df.columns else "UNKNOWN"
            if player.lower() == "travis hunter":
                pos = "WR"
                df["Pos"] = "WR"

            if pos not in {"QB","RB","WR","TE"}:
                print(f"    → skipping {player!r} with position '{pos}'")
                continue

            pos_groups.setdefault(pos, []).append(df)
            print(f"    ✔ collected stats for {player!r} under Pos='{pos}'")

        # 5) Flush CSVs for this year
        flush_pos_groups_to_csv(pos_groups, output_dir)

if __name__ == "__main__":
    main()


/Users/sethfried/Fantasy Football/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



=== Processing rookies for 2013 ===
Fetching draft page for 2013…
  Using column data-stat='college_link' for links
  Waiting 5.4s before fetching 'Luke Joeckel'
    → skipping 'Luke Joeckel' with position 'OL'
  Waiting 5.8s before fetching 'Dion Jordan'
    ✔ collected stats for 'Dion Jordan' under Pos='WR'
  Waiting 5.2s before fetching 'Lane Johnson'
    → skipping 'Lane Johnson' with position 'OL'
  Waiting 5.4s before fetching 'Ezekiel Ansah'
    → skipping 'Ezekiel Ansah' with position 'DL'
  Waiting 5.2s before fetching 'Barkevious Mingo'
    → skipping 'Barkevious Mingo' with position 'DL'
  Waiting 5.8s before fetching 'Jonathan Cooper'
    → skipping 'Jonathan Cooper' with position 'OL'
  Waiting 5.5s before fetching 'Tavon Austin'
    ✔ collected stats for 'Tavon Austin' under Pos='RB'
  Waiting 5.7s before fetching 'Dee Milliner'
    → skipping 'Dee Milliner' with position 'DB'
  Waiting 5.0s before fetching 'Chance Warmack'
    → skipping 'Chance Warmack' with position '